# Функции


## Функция удаления е-дуг

In [0]:
def e_arc_removal(data):
    for state in data.index[::-1]:
        if data.loc[state][2] != '':
            a = data.loc[state][2]
            if len(a) > 1:
                a = a.split(',')
            data.loc[state][2] = ''
            for i in a:
                for j in range(len(data.loc[i])):  
                    if data.loc[state][j] == '1':
                        break
                    if data.loc[state][j] != '' and data.loc[i][j] != '':
                        data.loc[state][j] = data.loc[state][j] + ',' 
                    data.loc[state][j] += data.loc[i][j]  
    data.drop(['e'], axis='columns', inplace=True)
    return data

## Функция объединения переходов

In [0]:
def union_conversions(data):
    data_new = pd.DataFrame(columns=[0, 1, 'Final'], index=['S'])
    data_new.loc['S'] = data.loc['S']
    a = list()
    for i in range(len(data_new.loc['S'])):
        if data.loc['S'][i] != '' and data.loc['S'][i] != '1':
            a.append(data.loc['S'][i])
    flag = False
    while flag == False:  
        if a[0] in data.index:
            data_new.loc[a[0]] = data.loc[a[0]]
            for i in range(len(data_new.loc[a[0]])):
                if data.loc[a[0]][i] != '' and data.loc[a[0]][i] != '1' and data.loc[a[0]][i] not in a and data.loc[a[0]][i] not in data_new.index:
                    a.append(data.loc[a[0]][i])
            a.pop(0)
        else:
            b = a[0]   
            ind = list(b.split(','))
            data_new.loc[b] = data.loc[ind].apply(','.join, axis=0)
            for i in range(len(data_new.loc[b])):
                c = data_new.loc[b][i].split(',')
                d = set()
                for j in range(len(c)):
                    if c[j] != '':
                        d.add(c[j])
                data_new.loc[b][i] = ','.join(d)
            if data_new.loc[b][-1] != '':
                data_new.loc[b][-1] = '1'        
            for i in range(len(data_new.loc[b])):
                if data_new.loc[b][i] != '' and data_new.loc[b][i] != '1' and data_new.loc[b][i] not in a and data_new.loc[b][i] not in data_new.index:
                    a.append(data_new.loc[b][i])
            a.pop(0)     
        if len(a) == 0:
            flag = True
    return data_new

## Функция удаления эквивалентных состояний

In [0]:
def drop_equal_state(data):
    data_new = pd.DataFrame(data)
    state_for_del = list()
    for state in data.index:
        for state2 in data_new.index:
            if state != state2:
                flag = 0
                for i in range(len(data_new.loc[state2])):
                    if data.loc[state][i] == data_new.loc[state2][i]:
                        flag += 1
                if flag == 3:
                    state_for_del.append(';'.join(list([state, state2])))
    for i in state_for_del:
        l = list(i.split(';'))
        if l[0] in data_new.index and l[1] in data_new.index:
            data_new.drop([l[1]], axis='index', inplace=True)
            for state_replace in data_new.index:
                for j in range(len(data_new.loc[state_replace])):
                    if data_new.loc[state_replace][j] == l[1]:
                        data_new.loc[state_replace][j] = l[0]
    return data_new

# Исходная строка

In [0]:
import pandas as pd

In [0]:
data  = [['', 'A', 'B', ''], ['', '', '', '1'], ['', 'A', 'Z', ''], ['', 'B', 'C', ''], ['D', '', '', ''], ['', 'E', '', ''], ['G', 'F', '', ''], ['', 'H', '', ''], ['', 'H', '', ''], ['K', 'J', 'Z', ''], ['', 'H', '', ''], ['', 'H', '', '']]
columns = [0, 1, 'e', 'Final']
index = ['S', 'Z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K']

In [122]:
data0 = pd.DataFrame(data, columns=columns, index=index)
data0

,0,1,e,Final
S,,A,B,
Z,,,,1
A,,A,Z,
B,,B,C,
C,D,,,
D,,E,,
E,G,F,,
F,,H,,
G,,H,,
H,K,J,Z,


In [123]:
data_new = e_arc_removal(data0)
data_new

,0,1,Final
S,D,"A,B",
Z,,,1
A,,A,1
B,D,B,
C,D,,
D,,E,
E,G,F,
F,,H,
G,,H,
H,K,J,1


In [124]:
data_new_0 = union_conversions(data_new) #Есть повторяющиеся состояния, но я от них избавлюсь на следующем шаге (А,В и В,А)
data_new_0

,0,1,Final
S,D,"A,B",
D,,E,
"A,B",D,"B,A",1
E,G,F,
"B,A",D,"B,A",1
G,,H,
F,,H,
H,K,J,1
K,,H,
J,,H,


In [125]:
drop_equal_state(data_new_0)

,0,1,Final
S,D,"A,B",
D,,E,
"A,B",D,"A,B",1
E,G,G,
G,,H,
H,G,G,1


# Строка 1

In [0]:
data  = [['', 'A', 'B', ''], ['', '', '', '1'], ['', 'A', 'Z', ''], ['', 'B', 'C', ''], ['D', '', '', ''], ['E', '', '', ''], ['F', 'G', '', ''], ['H', '', '', ''], ['', 'H', '', ''], ['J', 'K', 'Z', ''], ['H', '', '', ''], ['', 'H', '', '']]
columns = [0, 1, 'e', 'Final']
index = ['S', 'Z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K']

In [127]:
data1 = pd.DataFrame(data, columns=columns, index=index)
data1

,0,1,e,Final
S,,A,B,
Z,,,,1
A,,A,Z,
B,,B,C,
C,D,,,
D,E,,,
E,F,G,,
F,H,,,
G,,H,,
H,J,K,Z,


In [128]:
data1_new = e_arc_removal(data1)
data1_new

,0,1,Final
S,D,"A,B",
Z,,,1
A,,A,1
B,D,B,
C,D,,
D,E,,
E,F,G,
F,H,,
G,,H,
H,J,K,1


In [129]:
data_new_1 = union_conversions(data1_new) #Есть повторяющиеся состояния, но я от них избавлюсь на следующем шаге (А,В и В,А)
data_new_1

,0,1,Final
S,D,"A,B",
D,E,,
"A,B",D,"B,A",1
E,F,G,
"B,A",D,"B,A",1
F,H,,
G,,H,
H,J,K,1
J,H,,
K,,H,


In [130]:
drop_equal_state(data_new_1)

,0,1,Final
S,D,"A,B",
D,E,,
"A,B",D,"A,B",1
E,F,G,
F,H,,
G,,H,
H,F,G,1


# Строка 2

In [0]:
data  = [['', '', 'A', ''], ['', '', '', '1'], ['A', 'A', 'B', ''], ['C', 'D', '', ''], ['D', '', '', ''], ['', '', 'E', ''], ['E', 'E', 'Z', '']]
columns = [0, 1, 'e', 'Final']
index = ['S', 'Z', 'A', 'B', 'C', 'D', 'E']

In [132]:
data2 = pd.DataFrame(data, columns=columns, index=index)
data2

,0,1,e,Final
S,,,A,
Z,,,,1
A,A,A,B,
B,C,D,,
C,D,,,
D,,,E,
E,E,E,Z,


In [133]:
data2_new = e_arc_removal(data2)
data2_new

,0,1,Final
S,"A,C","A,D",
Z,,,1
A,"A,C","A,D",
B,C,D,
C,D,,
D,E,E,1
E,E,E,1


In [134]:
data_new_2 = union_conversions(data2_new) # Правильно!
data_new_2

,0,1,Final
S,"A,C","A,D",
"A,C","A,C,D","A,D",
"A,D","E,A,C","E,A,D",1
"A,C,D","E,A,C,D","E,A,D",1
"E,A,C","E,A,C,D","E,A,D",1
"E,A,D","E,A,C","E,A,D",1
"E,A,C,D","E,A,C,D","E,A,D",1


In [135]:
drop_equal_state(drop_equal_state(data_new_2))

,0,1,Final
S,"A,C","A,D",
"A,C","A,D","A,D",
"A,D","A,D","A,D",1


# Строка 3

In [0]:
data  = [['', '', 'A', ''], ['', '', '', '1'], ['C', 'B', 'D', ''], ['', 'A', '', ''], ['', 'A', '', ''], ['', '', 'E,F', ''], ['', 'E', 'Z', ''], ['', 'F', 'G', ''], ['H', '', '', ''], ['', 'Z', '', '']]
index = ['S', 'Z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [137]:
data3 = pd.DataFrame(data, columns=columns, index=index)
data3

,0,1,e,Final
S,,,A,
Z,,,,1
A,C,B,D,
B,,A,,
C,,A,,
D,,,"E,F",
E,,E,Z,
F,,F,G,
G,H,,,
H,,Z,,


In [138]:
data3_new = e_arc_removal(data3)
data3_new

,0,1,Final
S,"C,H","B,E,F",1
Z,,,1
A,"C,H","B,E,F",1
B,,A,
C,,A,
D,H,"E,F",1
E,,E,1
F,H,F,
G,H,,
H,,Z,


In [139]:
data_new_3 = union_conversions(data3_new) # Есть повторяющиеся состояния, но удалю потом как эквивалентные
data_new_3

,0,1,Final
S,"C,H","B,E,F",1
"C,H",,"Z,A",
"B,E,F",H,"E,A,F",1
"Z,A","H,C","B,E,F",1
H,,Z,
"E,A,F","H,C","B,E,F",1
"H,C",,"Z,A",
Z,,,1


In [140]:
drop_equal_state(drop_equal_state(data_new_3))

,0,1,Final
S,"C,H","B,E,F",1
"C,H",,S,
"B,E,F",H,S,1
H,,Z,
Z,,,1


# Строка 4

In [0]:
data  = [['B', '', 'A,C', ''], ['', '', '', '1'], ['A', 'A', 'Z', ''], ['Z', '', '', ''], ['D', 'E', 'Z', ''], ['', 'C', '', ''], ['C', '', '', '']]
index = ['S', 'Z', 'A', 'B', 'C', 'D', 'E']

In [142]:
data4 = pd.DataFrame(data, columns=columns, index=index)
data4

,0,1,e,Final
S,B,,"A,C",
Z,,,,1
A,A,A,Z,
B,Z,,,
C,D,E,Z,
D,,C,,
E,C,,,


In [143]:
data4_new = e_arc_removal(data4)
data4_new

,0,1,Final
S,"B,A,D","A,E",1
Z,,,1
A,A,A,1
B,Z,,
C,D,E,1
D,,C,
E,C,,


In [144]:
data_new_4 = union_conversions(data4_new) # Есть повторяющиеся состояния
data_new_4

,0,1,Final
S,"B,A,D","A,E",1
"B,A,D","Z,A","A,C",1
"A,E","A,C",A,1
"Z,A",A,A,1
"A,C","A,D","E,A",1
A,A,A,1
"A,D",A,"A,C",1
"E,A","A,C",A,1


In [145]:
drop_equal_state(drop_equal_state(drop_equal_state(data_new_4)))

,0,1,Final
S,"B,A,D","A,E",1
"B,A,D","Z,A",S,1
"A,E",S,"Z,A",1
"Z,A","Z,A","Z,A",1


# Строка 5

In [0]:
data  = [['C', 'A,B', '', ''], ['', '', '', '1'], ['', 'Z', '', ''], ['C', '', '', ''], ['', '', 'D', ''], ['', 'E', 'Z', ''], ['D', '', '', '']]
index = ['S', 'Z', 'A', 'B', 'C', 'D', 'E']

In [147]:
data5 = pd.DataFrame(data, columns=columns, index=index)
data5

,0,1,e,Final
S,C,"A,B",,
Z,,,,1
A,,Z,,
B,C,,,
C,,,D,
D,,E,Z,
E,D,,,


In [148]:
data5_new = e_arc_removal(data5)
data5_new

,0,1,Final
S,C,"A,B",
Z,,,1
A,,Z,
B,C,,
C,,E,1
D,,E,1
E,D,,


In [149]:
data_new_5 = union_conversions(data5_new) #Правильно!
data_new_5

,0,1,Final
S,C,"A,B",
C,,E,1
"A,B",C,Z,
E,D,,
Z,,,1
D,,E,1


In [150]:
drop_equal_state(data_new_5)

,0,1,Final
S,C,"A,B",
C,,E,1
"A,B",C,Z,
E,C,,
Z,,,1
